In [1]:
import os
from boxsdk import OAuth2, Client
import re
import warnings

from datetime import datetime, date
from uuid import uuid4
from dateutil.tz import tzlocal
from ast import literal_eval

import numpy as np
import pandas as pd
import warnings
import librosa
import pickle

from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.file import Subject
from pynwb.epoch import TimeIntervals
from pynwb.image import ImageSeries
from ndx_manoli_meta import AssayMetadata

from nwb_utils import get_date_from_block
from box_utils import *
from behavior_error_checks import *

In [27]:
# --- USER PARAMETERS ---

# metadata_files = ['metadata_intros_v4.csv','metadata_TMs.csv','metadata_ppts_v2.csv','metadata_reunions.csv','metadata_aggressions.csv']
# metadata_files = ['metadata_TMs.csv','metadata_ppts_v2.csv','metadata_reunions.csv','metadata_aggressions.csv']
metadata_files = ['metadata_ppts_v2.csv','metadata_reunions.csv','metadata_aggressions.csv']



lab = "Manoli @ UCSF"
excl = False # include all the assays
timeline = 'Matches Shank3 assays.'
# path to storage for NWB files
nwbfile_path = os.path.join('M:\\','scn2a-paper-GWJSNH','NWB-files')
# whether to write NWB files to disk yet
write_NWB_to_disk = True

for metadata_file in metadata_files:

    # load metadata
    meta = pd.read_csv(metadata_file)
    meta.FocalColor = meta.FocalColor.apply(literal_eval) # convert the colors to real arrays

    # -- loop over metadata
    for i, ptag in enumerate(meta.PairTag):
        assay_type = meta.AssayType[i]
        nwbfilename = f'{ptag}_{assay_type}.nwb'
        print(nwbfilename)

        # check if file already exists
        wfullpath = os.path.join(nwbfile_path,nwbfilename)    
        if not os.path.exists(wfullpath):    

            # get session specific metadata
            thisdate = str(meta.RecDate[i])
            pairdate = str(meta.PairDate[i])

            # calculate days post pairing
            rdate = date(int(thisdate[0:4]),int(thisdate[4:6]),int(thisdate[6:]))
            pdate = date(int(pairdate[0:4]),int(pairdate[4:6]),int(pairdate[6:]))
            dpp = rdate-pdate

            # set up recording time... it would be nice to get actual video data for the times
            datepieces = get_date_from_block(thisdate)
            rtime = meta.RecTime[i]
            timepieces = rtime.split(':')
            sess_start = datetime(datepieces[0],datepieces[1],datepieces[2],int(timepieces[0]),int(timepieces[1]),0,0,tzlocal())

            session_description = f'Behavioral annotations from Scn2a phenotyping pair {ptag} in a(n) {assay_type} assay.'

            # make NWB file
            nwbfile = NWBFile(
                session_description=session_description,
                identifier = str(uuid4()),
                session_start_time = sess_start,
                lab=lab,
                experimenter=meta.RanBy[i],
                session_id = nwbfilename[0:-4], # check this
            )

            # add subject info
            nwbfile.subject = Subject(
                subject_id = meta.FocalID[i],
                species = 'Microtus ochrogaster',
                sex = meta.FocalSex[i],
                genotype = meta.FocalGT[i]
            )

            # get assay duration
            dur = float(meta.AssayDuration[i])

            # figure out partner info
    #         if meta.FocalSex[i]=='F':
    #             pGT = meta.MaleGT[i]
    #         elif meta.FocalSex[i]=='M':

    #             pGT = meta.FemaleGT[i]
    #         else:
    #             print(f'Focal sex is neither F nor M; something is wrong with {ptag}.')
            pGT = 'WT'

            # calculate time since pairing
            days = meta.RecDate[i] - meta.PairDate[i]

            # add lab metadata

    #         room = meta.AssayRoom[i]
            room = '?'
            colors = meta.FocalColor[i][0]
    #         etho = meta.Ethogram[i]
            exp = meta.ScoredBy[i]
            timeline_comp = meta.FullTimeline[i]
            annotations = os.path.join(meta.ScorePath[i],meta.ScoreFile[i])
            dpostpair = int(dpp.days)

            match assay_type:
                case 'introduction':
                    metaObj = AssayMetadata(
                        assay_type=assay_type,
                        exclude_flag=excl,
                        duration=dur,
                        room=room,
                        timeline=timeline,
                        ethogram='JS_Scn2aX12022_Intro_RI_061124.boris',
                        scorer=exp,
                        timeline_complete=timeline_comp,
                        colors=colors,
                        assay_type__partner_GT=pGT,
                        assay_type__description='Introduction.',
                        assay_type__days_post_pairing=dpostpair,
                        assay_type__annotations=annotations
                    )


                case 'mating':
                    metaObj = AssayMetadata(
                        assay_type=assay_type,
                        exclude_flag=excl,
                        duration=dur,
                        room=room,
                        timeline=timeline,
                        ethogram='Scn2aX12022_TM_GW.boris',
                        scorer=exp,
                        timeline_complete=timeline_comp,
                        colors=colors,
                        assay_type__partner_GT=pGT,
                        assay_type__description='Timed mating.',
                        assay_type__days_post_pairing=dpostpair,
                        assay_type__annotations=annotations

                    )


                case 'PPT':

                    if meta.LeftType[i]=='Partner':
                        pchamb = 'Left'
                    else:
                        pchamb = 'Right'

                    metaObj = AssayMetadata(
                        assay_type=assay_type,
                        exclude_flag=excl,
                        duration=dur,
                        room=room,
                        timeline=timeline,
                        ethogram='Scn2aX12022_PPT_GW.boris',
                        scorer=exp,
                        timeline_complete=timeline_comp,
                        colors=colors,
                        assay_type__partner_GT=pGT,
                        assay_type__description='Linear three chamber tethered partner preference test.',
                        assay_type__days_post_pairing=dpostpair,
                        assay_type__stranger_GT='WT',
                        assay_type__PPT_lane=meta.PPTlane[i],
                        assay_type__partner_chamber=pchamb,
                        assay_type__left_GT=meta.LeftGT[i],
                        assay_type__left_sex=meta.LeftSex[i],
                        assay_type__right_GT=meta.RightGT[i],
                        assay_type__right_sex=meta.RightSex[i],
                        assay_type__left_type=meta.LeftType[i],
                        assay_type__right_type=meta.RightType[i],
                        assay_type__annotations=annotations                
                    )

                case 'reunion':
                    metaObj = AssayMetadata(
                        assay_type=assay_type,
                        exclude_flag=excl,
                        duration=dur,
                        room=room,
                        timeline=timeline,
                        ethogram='NK_CompiledObservations.boris',
                        scorer=exp,
                        timeline_complete=timeline_comp,
                        colors=colors,
                        assay_type__partner_GT=pGT,
                        assay_type__description='Separation/reunion.',
                        assay_type__days_post_pairing=dpostpair,
                        assay_type__isolation_length=60*60,
                        assay_type__annotations=annotations

                    )

                case 'aggression':
                    metaObj = AssayMetadata(
                        assay_type=assay_type,
                        exclude_flag=excl,
                        duration=dur,
                        room=room,
                        timeline=timeline,
                        ethogram='JS_Scn2aX12022_Intro_RI_061124.boris',
                        scorer=exp,
                        timeline_complete=timeline_comp,
                        colors=colors,
                        assay_type__stranger_GT='WT',
                        assay_type__description='Aggression towards an opposite-sex intruder to the homecage.',
                        assay_type__days_post_pairing=dpostpair,
                        assay_type__isolation_length=60*60,
                        assay_type__annotations=annotations

                    )

            # Add the test LabMetaDataExtensionExample to the NWBFile
            nwbfile.add_lab_meta_data(lab_meta_data=metaObj)

            # Add video file
            video_ext_file = ImageSeries(
                name='behaviorVideo',
                description='Raw original video.',
                unit='n.a.',
                external_file=[os.path.join(meta.VideoPath[i],meta.VideoFile[i])],
                format='external',
                starting_time=0.0,
                rate=25.0,
            )

            nwbfile.add_acquisition(video_ext_file)

            # --- Add annotated behavior data

            # Make sure these files are downloaded with the same path configuration as listed in the metadata
            # Or adjust here as I did with telling the code to go up a level
            scoretab = pd.read_csv(os.path.join('..',meta.ScorePath[i],meta.ScoreFile[i])) # load up csv of annotations

            # change problematic column names
            scoretab.rename(columns={'Start (s)':'start'}, inplace=True)
            scoretab.rename(columns={'Stop (s)':'end'}, inplace=True)
            scoretab.rename(columns={'Duration (s)':'duration'}, inplace=True)
            scoretab.rename(columns={'Behavior type':'behavior_type'}, inplace=True)

            # --- Clean up small gaps in the behavior table
            # this assumes the score files have already been reviewed for major problems; everything at this stage should pass

            cleantab = set_event_ends(scoretab)

            # make NWB object corresponding to the annotation table
            behavior_intervals = TimeIntervals(name="annotated_behavior",
                description="Intervals of scored behavior.")

            behavior_intervals.add_column(name="behavior", description="The annotation from the ethogram.")
            behavior_intervals.add_column(name="duration", description="Duration of the behavior.")
            behavior_intervals.add_column(name="atype", description="Point or state event.")

            # populate table
            for i, start in enumerate(cleantab.start):
                end = cleantab.end[i]
                behav = cleantab.Behavior[i]
                atype = cleantab.behavior_type[i]
                dur = cleantab.duration[i]
                behavior_intervals.add_row(start_time=start,stop_time=end,behavior=behav,atype=atype,duration=dur)

            # add to NWB file
            nwbfile.add_time_intervals(behavior_intervals)
            
                    # write file to disk
            if write_NWB_to_disk:
                with NWBHDF5IO(wfullpath, "w") as io:
                    io.write(nwbfile)

Nov22_Pair1_PPT.nwb


M:\utilities\analysis_utilities\behavior_error_checks.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stateonly.end.iloc[:-1] = round(stateonly.start.iloc[1:],3)-0.001
C:\Users\Nerissa\anaconda3\envs\voc230824\lib\site-packages\hdmf\build\objectmapper.py:260: DtypeConversionWarning: Spec 'vole_metadata/colors': Value with data type int32 is being converted to data type float32 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


Nov22_Pair15_PPT.nwb


M:\utilities\analysis_utilities\behavior_error_checks.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stateonly.end.iloc[:-1] = round(stateonly.start.iloc[1:],3)-0.001
C:\Users\Nerissa\anaconda3\envs\voc230824\lib\site-packages\hdmf\build\objectmapper.py:260: DtypeConversionWarning: Spec 'vole_metadata/colors': Value with data type int32 is being converted to data type float32 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


Nov22_Pair16_PPT.nwb


...

M:\utilities\analysis_utilities\behavior_error_checks.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stateonly.end.iloc[:-1] = round(stateonly.start.iloc[1:],3)-0.001
C:\Users\Nerissa\anaconda3\envs\voc230824\lib\site-packages\hdmf\build\objectmapper.py:260: DtypeConversionWarning: Spec 'vole_metadata/colors': Value with data type int32 is being converted to data type float32 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


Nov22_Pair7_aggression.nwb


M:\utilities\analysis_utilities\behavior_error_checks.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stateonly.end.iloc[:-1] = round(stateonly.start.iloc[1:],3)-0.001
C:\Users\Nerissa\anaconda3\envs\voc230824\lib\site-packages\hdmf\build\objectmapper.py:260: DtypeConversionWarning: Spec 'vole_metadata/colors': Value with data type int32 is being converted to data type float32 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


Nov22_Pair8_aggression.nwb


M:\utilities\analysis_utilities\behavior_error_checks.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stateonly.end.iloc[:-1] = round(stateonly.start.iloc[1:],3)-0.001
C:\Users\Nerissa\anaconda3\envs\voc230824\lib\site-packages\hdmf\build\objectmapper.py:260: DtypeConversionWarning: Spec 'vole_metadata/colors': Value with data type int32 is being converted to data type float32 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


In [32]:
# load metadata
metadata_file = 'metadata_TMs.csv'
meta = pd.read_csv(metadata_file)
meta.FocalColor = meta.FocalColor.apply(literal_eval) # convert the colors to real arrays

lab = "Manoli @ UCSF"
excl = False # include all the assays
timeline = 'Matches Shank3 assays.'
# path to storage for NWB files
nwbfile_path = os.path.join('M:\\','scn2a-paper-GWJSNH','NWB-files')
# whether to write NWB files to disk yet
write_NWB_to_disk = True

# -- loop over metadata
for i, ptag in enumerate(meta.PairTag):
    assay_type = meta.AssayType[i]
    nwbfilename = f'{ptag}_{assay_type}.nwb'
    print(nwbfilename)

    # check if file already exists
    wfullpath = os.path.join(nwbfile_path,nwbfilename)    
    if not os.path.exists(wfullpath):    

        # get session specific metadata
        thisdate = str(meta.RecDate[i])
        pairdate = str(meta.PairDate[i])

        # calculate days post pairing
        rdate = date(int(thisdate[0:4]),int(thisdate[4:6]),int(thisdate[6:]))
        pdate = date(int(pairdate[0:4]),int(pairdate[4:6]),int(pairdate[6:]))
        dpp = rdate-pdate

        # set up recording time... it would be nice to get actual video data for the times
        datepieces = get_date_from_block(thisdate)
        rtime = meta.RecTime[i]
        timepieces = rtime.split(':')
        sess_start = datetime(datepieces[0],datepieces[1],datepieces[2],int(timepieces[0]),int(timepieces[1]),0,0,tzlocal())

        session_description = f'Behavioral annotations from Scn2a phenotyping pair {ptag} in a(n) {assay_type} assay.'

        # make NWB file
        nwbfile = NWBFile(
            session_description=session_description,
            identifier = str(uuid4()),
            session_start_time = sess_start,
            lab=lab,
            experimenter=meta.RanBy[i],
            session_id = nwbfilename[0:-4], # check this
        )

        # add subject info
        nwbfile.subject = Subject(
            subject_id = meta.FocalID[i],
            species = 'Microtus ochrogaster',
            sex = meta.FocalSex[i],
            genotype = meta.FocalGT[i]
        )

        # get assay duration
        dur = float(meta.AssayDuration[i])

        # figure out partner info
#         if meta.FocalSex[i]=='F':
#             pGT = meta.MaleGT[i]
#         elif meta.FocalSex[i]=='M':

#             pGT = meta.FemaleGT[i]
#         else:
#             print(f'Focal sex is neither F nor M; something is wrong with {ptag}.')
        pGT = 'WT'

        # calculate time since pairing
        days = meta.RecDate[i] - meta.PairDate[i]

        # add lab metadata

#         room = meta.AssayRoom[i]
        room = '?'
        colors = meta.FocalColor[i][0]
#         etho = meta.Ethogram[i]
        exp = meta.ScoredBy[i]
        timeline_comp = meta.FullTimeline[i]
        annotations = os.path.join(meta.ScorePath[i],meta.ScoreFile[i])
        dpostpair = int(dpp.days)

        match assay_type:
            case 'introduction':
                metaObj = AssayMetadata(
                    assay_type=assay_type,
                    exclude_flag=excl,
                    duration=dur,
                    room=room,
                    timeline=timeline,
                    ethogram='JS_Scn2aX12022_Intro_RI_061124.boris',
                    scorer=exp,
                    timeline_complete=timeline_comp,
                    colors=colors,
                    assay_type__partner_GT=pGT,
                    assay_type__description='Introduction.',
                    assay_type__days_post_pairing=dpostpair,
                    assay_type__annotations=annotations
                )


            case 'timedMating':
                metaObj = AssayMetadata(
                    assay_type=assay_type,
                    exclude_flag=excl,
                    duration=dur,
                    room=room,
                    timeline=timeline,
                    ethogram='Scn2aX12022_TM_GW.boris',
                    scorer=exp,
                    timeline_complete=timeline_comp,
                    colors=colors,
                    assay_type__partner_GT=pGT,
                    assay_type__description='Timed mating.',
                    assay_type__days_post_pairing=dpostpair,
                    assay_type__annotations=annotations

                )


            case 'PPT':

                if meta.LeftType[i]=='Partner':
                    pchamb = 'Left'
                else:
                    pchamb = 'Right'

                metaObj = AssayMetadata(
                    assay_type=assay_type,
                    exclude_flag=excl,
                    duration=dur,
                    room=room,
                    timeline=timeline,
                    ethogram='Scn2aX12022_PPT_GW.boris',
                    scorer=exp,
                    timeline_complete=timeline_comp,
                    colors=colors,
                    assay_type__partner_GT=pGT,
                    assay_type__description='Linear three chamber tethered partner preference test.',
                    assay_type__days_post_pairing=dpostpair,
                    assay_type__stranger_GT='WT',
                    assay_type__PPT_lane=meta.PPTlane[i],
                    assay_type__partner_chamber=pchamb,
                    assay_type__left_GT=meta.LeftGT[i],
                    assay_type__left_sex=meta.LeftSex[i],
                    assay_type__right_GT=meta.RightGT[i],
                    assay_type__right_sex=meta.RightSex[i],
                    assay_type__left_type=meta.LeftType[i],
                    assay_type__right_type=meta.RightType[i],
                    assay_type__annotations=annotations                
                )

            case 'reunion':
                metaObj = AssayMetadata(
                    assay_type=assay_type,
                    exclude_flag=excl,
                    duration=dur,
                    room=room,
                    timeline=timeline,
                    ethogram='NK_CompiledObservations.boris',
                    scorer=exp,
                    timeline_complete=timeline_comp,
                    colors=colors,
                    assay_type__partner_GT=pGT,
                    assay_type__description='Separation/reunion.',
                    assay_type__days_post_pairing=dpostpair,
                    assay_type__isolation_length=60*60,
                    assay_type__annotations=annotations

                )

            case 'aggression':
                metaObj = AssayMetadata(
                    assay_type=assay_type,
                    exclude_flag=excl,
                    duration=dur,
                    room=room,
                    timeline=timeline,
                    ethogram='JS_Scn2aX12022_Intro_RI_061124.boris',
                    scorer=exp,
                    timeline_complete=timeline_comp,
                    colors=colors,
                    assay_type__stranger_GT='WT',
                    assay_type__description='Aggression towards an opposite-sex intruder to the homecage.',
                    assay_type__days_post_pairing=dpostpair,
                    assay_type__isolation_length=60*60,
                    assay_type__annotations=annotations

                )

        # Add the test LabMetaDataExtensionExample to the NWBFile
        nwbfile.add_lab_meta_data(lab_meta_data=metaObj)

        # Add video file
        video_ext_file = ImageSeries(
            name='behaviorVideo',
            description='Raw original video.',
            unit='n.a.',
            external_file=[os.path.join(meta.VideoPath[i],meta.VideoFile[i])],
            format='external',
            starting_time=0.0,
            rate=25.0,
        )

        nwbfile.add_acquisition(video_ext_file)

        # --- Add annotated behavior data

        # Make sure these files are downloaded with the same path configuration as listed in the metadata
        # Or adjust here as I did with telling the code to go up a level
        scoretab = pd.read_csv(os.path.join('..',meta.ScorePath[i],meta.ScoreFile[i])) # load up csv of annotations

        # change problematic column names
        scoretab.rename(columns={'Start (s)':'start'}, inplace=True)
        scoretab.rename(columns={'Stop (s)':'end'}, inplace=True)
        scoretab.rename(columns={'Duration (s)':'duration'}, inplace=True)
        scoretab.rename(columns={'Behavior type':'behavior_type'}, inplace=True)

        # --- Clean up small gaps in the behavior table
        # this assumes the score files have already been reviewed for major problems; everything at this stage should pass

        cleantab = set_event_ends(scoretab)

        # make NWB object corresponding to the annotation table
        behavior_intervals = TimeIntervals(name="annotated_behavior",
            description="Intervals of scored behavior.")

        behavior_intervals.add_column(name="behavior", description="The annotation from the ethogram.")
        behavior_intervals.add_column(name="duration", description="Duration of the behavior.")
        behavior_intervals.add_column(name="atype", description="Point or state event.")

        # populate table
        for i, start in enumerate(cleantab.start):
            end = cleantab.end[i]
            behav = cleantab.Behavior[i]
            atype = cleantab.behavior_type[i]
            dur = cleantab.duration[i]
            behavior_intervals.add_row(start_time=start,stop_time=end,behavior=behav,atype=atype,duration=dur)

        # add to NWB file
        nwbfile.add_time_intervals(behavior_intervals)

                # write file to disk
        if write_NWB_to_disk:
            with NWBHDF5IO(wfullpath, "w") as io:
                io.write(nwbfile)

Nov22_Pair1_timedMating.nwb


M:\utilities\analysis_utilities\behavior_error_checks.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stateonly.end.iloc[:-1] = round(stateonly.start.iloc[1:],3)-0.001
C:\Users\Nerissa\anaconda3\envs\voc230824\lib\site-packages\hdmf\build\objectmapper.py:260: DtypeConversionWarning: Spec 'vole_metadata/colors': Value with data type int32 is being converted to data type float32 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


Nov22_Pair15_timedMating.nwb


M:\utilities\analysis_utilities\behavior_error_checks.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stateonly.end.iloc[:-1] = round(stateonly.start.iloc[1:],3)-0.001
C:\Users\Nerissa\anaconda3\envs\voc230824\lib\site-packages\hdmf\build\objectmapper.py:260: DtypeConversionWarning: Spec 'vole_metadata/colors': Value with data type int32 is being converted to data type float32 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


Nov22_Pair16_timedMating.nwb


...

M:\utilities\analysis_utilities\behavior_error_checks.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stateonly.end.iloc[:-1] = round(stateonly.start.iloc[1:],3)-0.001
C:\Users\Nerissa\anaconda3\envs\voc230824\lib\site-packages\hdmf\build\objectmapper.py:260: DtypeConversionWarning: Spec 'vole_metadata/colors': Value with data type int32 is being converted to data type float32 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


Nov22_Pair7_timedMating.nwb


M:\utilities\analysis_utilities\behavior_error_checks.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stateonly.end.iloc[:-1] = round(stateonly.start.iloc[1:],3)-0.001
C:\Users\Nerissa\anaconda3\envs\voc230824\lib\site-packages\hdmf\build\objectmapper.py:260: DtypeConversionWarning: Spec 'vole_metadata/colors': Value with data type int32 is being converted to data type float32 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


Nov22_Pair8_timedMating.nwb


M:\utilities\analysis_utilities\behavior_error_checks.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stateonly.end.iloc[:-1] = round(stateonly.start.iloc[1:],3)-0.001
C:\Users\Nerissa\anaconda3\envs\voc230824\lib\site-packages\hdmf\build\objectmapper.py:260: DtypeConversionWarning: Spec 'vole_metadata/colors': Value with data type int32 is being converted to data type float32 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)


In [31]:
nwbfile

,start_time,stop_time,behavior,duration,atype
id,,,,,
0,90.722,90.722,Start assay,NaN,POINT
1,90.723,101.401,No interaction,10.678,STATE
2,101.402,101.902,Investigate,0.500,STATE
3,101.903,107.904,No interaction,6.001,STATE


In [ ]:
# ----- Generate metadata object based on metadata read in from table -----

def make_metadata_object(atype,excl,dur,room,timeline,etho,exp,timeline_comp,
                         colors,pGT,dpostpair,annotations,sID=None,sGT=None,lane=None,chamber=None,
                        leftGT=None,leftSex=None,rightGT=None,rightSex=None,leftType=None,rightType=None):

        
    match atype:
        case 'introduction':
            metaObj = AssayMetadata(
                assay_type=atype,
                exclude_flag=excl,
                duration=dur,
                room=room,
                timeline=timeline,
                ethogram=etho,
                scorer=exp,
                timeline_complete=timeline_comp,
                colors=colors,
                assay_type__partner_GT=pGT,
                assay_type__description='Introduction.',
                assay_type__days_post_pairing=dpostpair,
                assay_type__annotations=annotations
            )

            
        case 'mating':
            metaObj = AssayMetadata(
                assay_type=atype,
                exclude_flag=excl,
                duration=dur,
                room=room,
                timeline=timeline,
                ethogram=etho,
                scorer=exp,
                timeline_complete=timeline_comp,
                colors=colors,
                assay_type__partner_GT=pGT,
                assay_type__description='Timed mating.',
                assay_type__days_post_pairing=dpostpair,
                assay_type__annotations=annotations

            )
            
            
        case 'PPT':
            metaObj = AssayMetadata(
                assay_type=atype,
                exclude_flag=excl,
                duration=dur,
                room=room,
                timeline=timeline,
                ethogram=etho,
                scorer=exp,
                timeline_complete=timeline_comp,
                colors=colors,
                assay_type__partner_ID=pID,
                assay_type__partner_GT=pGT,
                assay_type__description='Linear three chamber tethered partner preference test.',
                assay_type__days_post_pairing=dpostpair,
                assay_type__stranger_GT=sGT,
                assay_type__PPT_lane=lane,
                assay_type__partner_chamber=chamber,
                assay_type__leftGT=leftGT,
                assay_type__leftSex=leftSex,
                assay_type__rightGT=rightGT,
                assay_type__rightSex=rightSex,
                assay_type__leftType=leftType,
                assay_type__rightType=rightType,
                assay_type__annotations=annotations                
            )
            
        case 'reunion':
            metaObj = AssayMetadata(
                assay_type=atype,
                exclude_flag=excl,
                duration=dur,
                room=room,
                timeline=timeline,
                ethogram=etho,
                scorer=exp,
                timeline_complete=timeline_comp,
                colors=colors,
                assay_type__partner_GT=pGT,
                assay_type__description='Separation/reunion.',
                assay_type__days_post_pairing=dpostpair,
                assay_type__annotations=annotations
            
            )
            
        case 'aggression':
            metaObj = AssayMetadata(
                assay_type=atype,
                exclude_flag=excl,
                duration=dur,
                room=room,
                timeline=timeline,
                ethogram=etho,
                scorer=exp,
                timeline_complete=timeline_comp,
                colors=colors,
                assay_type__stranger_GT=sGT,
                assay_type__description='Aggression towards an opposite-sex intruder to the homecage.',
                assay_type__days_post_pairing=dpostpair,
                assay_type__annotations=annotations
            
            )
            
    return metaObj